# Visualize and analyze data

## Package imports

In [ ]:
import sys
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} laspy
import laspy
import numpy as np

sys.path.append("..")
from pytreedb import db

## Import database

Define (local) MongoDB connection and import database from URL

In [ ]:
mydbfile = "my_complex_query_pytree.db"
db_url = "https://github.com/3dgeo-heidelberg/pytreedb/raw/main/data/test/data.db"
mydb = db.PyTreeDB(dbfile=mydbfile)
mydb.import_db(db_url, overwrite=True)

## Show data on a map

See `query_by_geometry.ipynb`

## Bar plots and Histograms

Available trees for each data source

Histograms of tree height/DBH/etc.

## Venn diagram?

## Scatter plot of metrics

Plot DBH against tree height

Compare crown diameter to tree height ratio for different species

Scatter plots of metrics obtained from different sources

## 3D plots of point clouds

In [ ]:
# pypotree / PPTK / Open3D / PyntCloud / matplotlib ?